<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/Szablon-wczytywania-danych-w-sparku/colabs/Wczytywanie_danych_NASA_w_sparku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wczytywanie danych w sparku

Utworzenie środowiska pyspark do obliczeń:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

!pip install -q findspark

import findspark
findspark.init()

Utowrzenie sesji:

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Połączenie z dyskiem:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Wczytanie danych NASA znajdujących się na dysku w sparku:

In [4]:
df = spark.read.format('csv').option("header", True).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')
df.show(5)

+---------+-------+------+----------------+----------------+----------------+-----------------+----------------+----------------+-----------------+---+-----+-----+----------------+---+-----------------+---+----------------+----------------+---+---------+--------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-----------------+----------------+-----------------+-------+-------------------+-------------------+--------------------+-----------------+-----------------+-----------------+-----------------+----------------+-------------------+----------------+-----------------+----------+
|      lon|    lat|  Date|          SWdown|          LWdown|           SWnet|            LWnet|             Qle|              Qh|               Qg| Qf|Sn

Konwersja typu danych:

In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType
for column in df.columns:
    if column == "Date":
        df = df.withColumn(column, col(column).cast(IntegerType()))
    else:
        df = df.withColumn(column, col(column).cast(FloatType()))


Zanim zacznimy pisać kwerendy należy jeszcze dodać nasz DataFrame (df) do "przestrzeni nazw tabel" Sparka:

In [6]:
df.createOrReplaceTempView("df")

Test:

In [7]:
%%time
spark.sql("""
          SELECT
          MIN(Date) AS min_date,
          MAX(Date) AS max_date,
          COUNT(DISTINCT Date) AS count_distinct_date
          FROM df
          """).show()

+--------+--------+-------------------+
|min_date|max_date|count_distinct_date|
+--------+--------+-------------------+
|  200001|  202309|                285|
+--------+--------+-------------------+

CPU times: user 1.16 s, sys: 153 ms, total: 1.31 s
Wall time: 3min 47s
